## Fine-tuning 할 LLaMA불러오기

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
model = AutoModelForCausalLM.from_pretrained("/home/wooseok/llama-7b-hf", device_map="auto", load_in_8bit=True)

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained("/home/wooseok/llama-7b-hf")


/home/wooseok/miniconda3/envs/mh/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /home/wooseok/miniconda3/envs/mh/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 6.1
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /home/wooseok/miniconda3/envs/mh/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda113_nocublaslt.so...


/home/wooseok/miniconda3/envs/mh/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)
Loading checkpoint shards: 100%|████████████████████████████████████████████████████████| 33/33 [00:18<00:00,  1.81it/s]
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [9]:
tokenizer.decode(model.generate(torch.tensor([[4066,2]]))[0])

' interest'

# 데이터셋 준비

In [2]:
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, txt_list, tokenizer):
        self.input_ids = []
        self.attention_masks = []

        for txt in txt_list:
            inputs = tokenizer(txt, truncation=True, max_length=512, padding='max_length')
            self.input_ids.append(inputs['input_ids'])
            self.attention_masks.append(inputs['attention_mask'])

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attention_masks[idx]


## 훈련인자 설정 및 Fine-tuning

In [3]:
from transformers import TrainingArguments,Trainer

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    do_train=True,
    do_eval=True,
    evaluation_strategy="steps",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()

<frozen importlib._bootstrap>:219: RuntimeWarning: pyarrow._fs.FileInfo size changed, may indicate binary incompatibility. Expected 64 from C header, got 88 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: pyarrow._fs.FileSelector size changed, may indicate binary incompatibility. Expected 48 from C header, got 72 from PyObject


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:14                                                                                   │
│                                                                                                  │
│   11 trainer = Trainer(                                                                          │
│   12 │   model=model,                                                                            │
│   13 │   args=training_args,                                                                     │
│ ❱ 14 │   train_dataset=train_dataset,                                                            │
│   15 │   eval_dataset=val_dataset                                                                │
│   16 )                                                                                           │
│   17                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'train_dataset' is not defined